## 앙상블 학습
- 앙상블 학습의 유형은 보팅, 배깅, 부스팅 세 가지로 나눌 수 있으며 이외에도 스태깅을 포함한 다양한 앙상블 방법이 있다.
- 보팅의 경우 서로 닫른 알고리즘을 가진 분류기를 결합하는 것이고 배깅의 경우 각각의 분류기가 모두 같은 유형의 알고리즘 기반이다.
- 정형 데이터의 예측 분석 영역에서는 매우 높은 에측 성능, Bagging과 Boosting
- 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행 시간, 유연성 등으로 애용.
- 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간단축시킨 XgBoost, LightGBM이 정형 데이터의 분류 영역에서 활용도 확대
- 앙상블의 앙상블이라고 불리는 스태깅 기법
- 앙상블의 기본 알고리즘은 결정 트리

#### Votinc Classifier
- 하드 보팅 : 다수결 원칙, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- 소프트보팅: 분류기들의 레이블 값 결정 확률을 모두 더해서 평균하고 이들 중 가장 높은 레이블 값을 최종 보팅 결과값으로 선정

In [5]:
import pandas as pd 
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression #일반선형모델에서 확률분포를 이항분포로 전제해서 함. 2진분류
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings 
warnings.filterwarnings('ignore')

cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [6]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), 
                                    ('KNN',knn_clf)], voting='soft')
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    test_size=0.2,
                                                    random_state=156)

# 개별 모델의 학습/예측/효과 학습용 데이터가 2개이니까 for문을 써서 해보기 
classifiers = [lr_clf, knn_clf]
for classifier in classifiers :
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test) 
    class_name =classifier.__class__.__name__ #classifiter의 class 이름
    print('{0} 정확도 : {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))


LogisticRegression 정확도 : 0.9386
KNeighborsClassifier 정확도 : 0.9035


In [12]:
# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred1 = vo_clf.predict(X_test)
class_name =classifier.__class__.__name__
print('{0} 정확도 : {1:.4f}'.format(class_name,accuracy_score(y_test, pred1)))
print(accuracy_score(y_test,pred1))

KNeighborsClassifier 정확도 : 0.9474
0.9473684210526315


In [13]:
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 세트와 테스트 피처 데이터를 DataFrame으로 로딩, 칼럼명은 feature_name 적용
    # 우선 칼럼명 없이 데이터프레임으로 불러온 후 칼럼명을 추가함
    X_train = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/human_activity/train/X_train.txt', sep='\s+', header=None)
    X_train.columns = feature_name
    X_test = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/human_activity/test/X_test.txt', sep='\s+',header=None)
    X_test.columns = feature_name
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/human_activity/train/y_train.txt', sep='\s+', header=None, names = ['action'])
    y_test = pd.read_csv('/Users/joy/Documents/GitHub/0Oong/수업내용/dataset/human_activity/test/y_test.txt', sep='\s+', header=None, names = ['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = get_human_dataset()

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test) 
accuracy = accuracy_score(y_test, pred)
class_name = rf_clf.__class__.__name__
print('{} 정확도 : {:.4f}'.format(class_name, accuracy))

RandomForestClassifier 정확도 : 0.9247


## GBM(Gradient Boosting Machine)
- 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
- 가중치 업데이트를 경사 하강법(Gradient Descent)를 이용한다.
- 분류는 물론이고 회귀도 가능
- 파라미터 : n_estimators(트리의 개수), max_depth(트리의 깊이), max_features(트리를 만들 때 최대의 feature의 개수)
    - loss : 경사하강법에서 사용할 비용함수 지정, 기본값, deviance 적용
    - learning_rate : GBM이 학습할 때마다 적용할 학습률. 오류값 보정 시 적용하는 계수로 0~1 사이의 값 지정. 기본값은 0.1 작게 설정하면 예측성능이 높아지나 수행시간이 오래 걸리고 큰 값을 적용하면 예측 성능이 떨어질 가능성이 높으나 빠른 수행이 가능하다. n_estimator와 상호 보완적으로 조합해 사용한다.
- n_estimator : weak learner의 개수
- subsample : weak learner가 학습에 사용하는 데이터의 샘플링 비율, 기본값은 1

n_estimators, learning_rate를 각각 크게, 작게 설정하면 에측률은 높아질수도 있겠지만 매우 오래걸릴 것임. 

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time 

X_train, y_train, X_test, y_test = get_human_dataset()

# 수행시간
# start_time = time.time()
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(gb_accuracy)

ValueError: Found input variables with inconsistent numbers of samples: [7352, 2947]

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time

X_train,X_test,y_train,y_test = get_human_dataset()
start_time = time.time()


gb_clf=GradientBoostingClassifier(random_state=0)

gb_clf.fit(X_train,y_train)
pred = gb_clf.predict(X_test)

accuracy = accuracy_score(y_test,pred)

print('GBM 정확도 : {0:4f}'.format(accuracy))
print('GBM 수행시간 : {0:1f} 초'.format(time.time()-start_time))

GBM 정확도 : 0.939260
GBM 수행시간 : 686.639074 초


In [28]:
# 교차검증 + 파라미터도 바꾸고 아무튼 다양한거를 시뮬레이션 하면서 한 번에 설정하는데 시간이 굉장히 오래걸림
# **주의** 부스팅 방식이랑 같이 사용하면 최소 30분~1시간이 걸림
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100,200],
        'learning_rate' : [0.05, 0.1],
        }
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1) #param은 2개씩 총 4개, cv까지 해서 4*2 = 8개
grid_cv.fit(X_train, y_train) #손실이 가장 낮은 최적의 파라미터를 구해줌

print('최적 하이퍼 파라미터 :\n', grid_cv.best_params_) #파라미터는 정해주는 것 계수랑 절편. 하이퍼파라미터는 내가 정해주는 것
print('최고 예측 정확도 : {:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


KeyboardInterrupt: 

In [ ]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도 : {:.4f}'.format(gb_accuracy))